In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# First we need to apply Bianrized Naive Byaes because word occurence matter smore
# Then we look at ngram features and how THAT affects our accuracy
# Then we look at TF-IDF representation

In [3]:
# import data
train_df = pd.read_csv("all/train.csv")

In [4]:
test_df = pd.read_csv("all/test.csv")

In [108]:
train_df[traid_df.toxic == 1].shape[0]/traid_df.shape[0]

0.09584448302009764

In [109]:
# so our train contains almost 0.1 percent of toxic comments

# A little preprocessing

In [110]:
# Lets first remove the features we dont require
train_df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [111]:
train_df.drop(['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis = 1, inplace=True)

In [112]:
test_df.drop('id', axis = 1, inplace=True)

In [113]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [123]:
veczr = CountVectorizer()
train_counts = veczr.fit_transform(list(train_df.comment_text.values))

In [124]:
train_counts.shape
vocab_count = train_counts.shape[1]

In [125]:
# The count for "explanation" in all documents
veczr.

62514

In [126]:
tfidf_transformer = TfidfTransformer()

In [127]:
tfidf_count = tfidf_transformer.fit_transform(train_counts)

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [128]:
clf = MultinomialNB().fit(tfidf_count, train_df.toxic)

In [129]:
tfidf_text_count = tfidf_transformer.transform(veczr.transform(list(test_df.comment_text.values)))
pred_y = clf.predict(tfidf_text_count)

In [130]:
test_y = pd.read_csv("all/test_labels.csv")

57888

In [ ]:
test_df

In [136]:
# Write my own Naivebayes
# With ngram and TFIDF

In [5]:
train_df.drop(['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis = 1, inplace=True)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [14]:
term_document = CountVectorizer(ngram_range=(1,2), max_features=1000000)

In [15]:
vocab = term_document.fit_transform(train_df.comment_text)

In [60]:
help(term_document.vocabulary)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).



In [17]:
tfidf = TfidfTransformer()

In [19]:
reweighted_vocabular = tfidf.fit_transform(vocab);

In [30]:
# Reweighted vocabulary now contains the transformed likelihoods of each ngram occuring.
nb_classifier = MultinomialNB()

In [31]:
nb_classifier.fit(reweighted_vocabular, y=train_df.toxic)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
test_labels = pd.read_csv("all/test_labels.csv")

In [56]:
# Assuming that the rest of all the labels are in order then
test_vocab = term_document.fit(test_df.comment_text)
test_reweighted_vocab = tfidf.fit(test_vocab)

TypeError: no supported conversion for types: (dtype('O'),)

# Making Document Term Matrix

In [87]:
train_x = train_df.comment_text[:50000]
train_y = train_df.toxic[:50000]
test_x = train_df.comment_text[50001:60000]
test_x = train_df.toxic[50001:60000]

In [88]:
vectorizer = CountVectorizer(ngram_range=(1,2), max_features=100000, stop_words='english')

In [89]:
dt_matrix = vectorizer.fit_transform(train_x)

## Now that we have the sparse representation lets turn to creating a dense representation and reweighting it to be TF-IDF representation

In [92]:
tfidf = TfidfTransformer()
reweighted_vocab = tfidf.fit_transform(dt_matrix)

In [118]:
P_toxic = 0
P_ntoxic = 0
def getPriors(data):
    nt = np.count_nonzero(data == 1)
    nnt = data[data == 0].shape[0]
    
    return nt, nnt, nt/data.shape[0], nnt/data.shape[0]

n_toxic, n_ntoxic, P_toxic, P_ntoxic = getPriors(train_y)
 

0.90234

In [170]:
"""Returns the likelihood of a word against the given type of document using TFIDF reweighting
"""

def likelihood(data_y, word, label, veczr, vocab):
#     So P(word|label) i.e. what is the likelihood of this word occuring for the given label
#     For this we use our reweighted vobaulary
#     Th elikelihood of the word can be calculated by adding columns where class = label
#     divide that by the total frequency of documents of type label
    
    likelihood = 0
    
    try:
        word_index = veczr.vocabulary_[word]
        for i in range(0, data_y.shape[0]):
            # here we take the dense version and access the current words TF-idf
            if (data_y[i] == label):
                dense_buff = vocab[i].todense()
                likelihood += dense_buff[0, word_index]
    except KeyError as e:
        word_index = 0
        
    
    return 1 + (likelihood/(n_toxic if label==1 else n_ntoxic))


In [165]:
# here we see the P('time' | toxic = 1)
print(likelihood(train_y, "fuck", 1, vectorizer, reweighted_vocab))

1.04288162484


In [178]:
def MAP(document, Pt, Pnt):
#     doc_sparse = vectorizer.transform(document)
#     doc_dense = doc_sparse.todense()

#   here we have all the features for our document
    features = document.split()
    bigram = [' '.join(features[i:i+2]) for i in range(len(features) - 2 + 1)]
    features.extend(bigram)
    
    
    prod_t = 1
    prod_nt = 1
    for feature in features:
        prod_t *= likelihood(train_y, feature, 1, vectorizer, reweighted_vocab)
        prod_nt *= likelihood(train_y, feature, 0, vectorizer, reweighted_vocab)
        
        
    vnb_toxic =  prod_t * Pt
    vnb_nottoxic = prod_nt * Pnt

    print("Not toxic: ", vnb_toxic)
    print("Toxic: ", vnb_nottoxic)
    
    fin_class = 0 if () else 1
    return fin_class
    

In [179]:
print(MAP("Fuck this shit", P_toxic, P_ntoxic))

Not toxic:  0.0993121196128
Toxic:  0.902479836955
1
